## image transfer

In [0]:
import numpy as np
import pickle
import scipy

import theano
import theano.tensor as T

import matplotlib.pyplot as plt
%matplotlib inline

import dlp_layers
from dlp_layers import floatX

from PIL import Image

In [0]:
from dlp_layers import softmax
from dlp_layers import InputLayer, ConvLayer, PoolLayer
IMAGE_W = 600
def build_model():
    net = {}
    net['input'] = InputLayer((1, 3, IMAGE_W, IMAGE_W))
    net['conv1_1'] = ConvLayer(net['input'], 64, 3)
    net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3)   
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(net['pool1'], 128, 3)
    net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(net['pool2'], 256, 3)
    net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3)
    net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3)
    net['conv3_4'] = ConvLayer(net['conv3_3'], 256, 3)
    net['pool3'] = PoolLayer(net['conv3_4'], 2)
    net['conv4_1'] = ConvLayer(net['pool3'], 512, 3)
    net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3)
    net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3)
    net['conv4_4'] = ConvLayer(net['conv4_3'], 512, 3)
    net['pool4'] = PoolLayer(net['conv4_4'], 2)
    net['conv5_1'] = ConvLayer(net['pool4'], 512, 3)
    net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3)
    net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3)
    net['conv5_4'] = ConvLayer(net['conv5_3'], 512, 3)
    net['pool5'] = PoolLayer(net['conv5_4'], 2)

    return net

In [0]:
net = build_model()
values = pickle.load(open('vgg19_normalized.pkl'))['param values']
dlp_layers.set_all_param_values(net['pool5'], values)

In [0]:
def processing_image(im, Image_Dimension = 600):
    img = im.resize( (Image_Dimension, Image_Dimension), Image.ANTIALIAS)
    original_image = img.copy()
    img = np.array(img).astype('float32')    
    img = np.rollaxis(np.rollaxis(img, 2, 1), 1, 0)
    ig = img.copy()
    for i in xrange(int(ig.shape[0])):
        img[i,:,:] = ig[int(ig.shape[0])-1-i,:,:]
    mean = np.array([np.mean(img[i,:,:]) for i in xrange(3)]).reshape(3,1,1)
    
    img -= mean
    img = img[np.newaxis]
    
    return original_image, img, mean

In [0]:
photo = Image.open('picture.jpg')
original_image, photo, m_pircure  = processing_image(photo)
plt.imshow(original_image)

In [0]:
art = Image.open('art.jpg')
rawim, art, mean_art = processing_image(art)
plt.imshow(rawim)

In [0]:
def reveal(x, MEAN_VALUES = mean_art):
    x = np.copy(x[0])
    x += MEAN_VALUES

    x = x[::-1]
    x = np.rollaxis(np.rollaxis(x,1,0),2,1)
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [0]:
all_layers = ['conv1_1', 'conv1_2','conv2_1','conv2_2', 'conv3_1', 'conv3_2', 'conv3_3', 'conv3_4',
          'conv4_1', 'conv4_2', 'conv4_3', 'conv4_4','conv5_1', 'conv5_2','conv5_3', 'conv5_4']
layers = {k: net[k] for k in all_layers}

In [0]:
input_im_theano = T.tensor4()
outputs = dlp_layers.get_output(layers.values(), input_im_theano)

photo_features = {k: theano.shared(output.eval({input_im_theano: photo}))
                  for k, output in zip(layers.keys(), outputs)}
art_features = {k: theano.shared(output.eval({input_im_theano: art}))
                for k, output in zip(layers.keys(), outputs)}

In [0]:
generated_image = theano.shared(floatX(np.random.uniform(-128, 128, (1, 3, IMAGE_W, IMAGE_W))))

gen_features = dlp_layers.get_output(layers.values(), generated_image)
gen_features = {k: v for k, v in zip(layers.keys(), gen_features)}

In [0]:
def content_loss(p, x):
    return 1.0/2 * ((p - x)**2).sum()

def style_loss(a, x):
    A = T.tensordot(a.flatten(ndim=3), a.flatten(ndim=3), axes = ([2],[2]))
    G = T.tensordot(x.flatten(ndim=3), x.flatten(ndim=3), axes = ([2],[2]))
    N = a.shape[1]
    M = a.shape[2] * a.shape[3]
    
    return 1.0/(4 * N**2 * M**2) * ((G - A)**2).sum()

def reconstruction(content_layers, style_layers, alpha = 1, beta = 1):

    L_c, L_s = 0, 0
    w = 1.0/len(style_layers)
    for i in xrange(len(content_layers)):
        L_c += content_loss(photo_features[content_layers[i]], gen_features[content_layers[i]])
        
    for i in xrange(len(style_layers)):
        L_s += w*style_loss(art_features[style_layers[i]], gen_features[style_layers[i]])
        
    total_loss = alpha*L_c + beta*L_s 
    grad = T.grad(total_loss, generated_image)    
       
    f_loss = theano.function([], total_loss)
    f_grad = theano.function([], grad)

    generated_image.set_value(floatX(np.random.uniform(-128, 128, (1, 3, IMAGE_W, IMAGE_W))))

    x0 = generated_image.get_value().astype('float64')
    xs = []
    xs.append(x0)

    learning_rate,gamma, velocity= 0.5, 0.3, 0
    for i in range(2001):
        generated_image.set_value(floatX(x0))
        
        grad = T.grad(total_loss, generated_image)
        f_loss = theano.function([], total_loss)
        f_grad = theano.function([], grad)

        dloss = np.array(f_grad()).astype('float64')
        velocity = gamma * velocity + learning_rate * dloss
        x0 -= velocity
        if i % 200 == 0:
            xs.append(x0)

    
    return xs[-1]



In [0]:
content_layers = ['conv4_2']
style_layers = ['conv1_1']
alpha = 0.001
beta = [1e6, 1e5, 1e4, 1e3]
images = []
for i in xrange(4):
    images.append(reconstruction(content_layers, style_layers, alpha = alpha, beta = beta[i]))
    
titles = ['10^-9', '10^-8', '10^-7','10^-6']
plt.figure(figsize=(60,60))
for i in xrange(4):
    plt.subplot(1, 4, i+1)
    plt.ylabel('{0}'.format(style_layers[-1]))
    plt.title('{0}'.format(titles[i]))
    plt.imshow(reveal(images[i]))
plt.savefig('{0}.jpg'.format(style_layers[-1]))

In [0]:
content_layers = ['conv4_2']
style_layers = ['conv1_1', 'conv2_1']
alpha = 0.001
beta = [2e6, 2e5, 2e4, 2e3]
images = []
for i in xrange(4):
    images.append(reconstruction(content_layers, style_layers, alpha = alpha, beta = beta[i]))
    
titles = ['10^-9', '10^-8', '10^-7','10^-6']
plt.figure(figsize=(60,60))
for i in xrange(4):
    plt.subplot(1, 4, i+1)
    plt.ylabel('{0}'.format(style_layers[-1]))
    plt.title('{0}'.format(titles[i]),)
    plt.imshow(reveal(images[i]))
plt.savefig('{0}.jpg'.format(style_layers[-1]))

In [0]:
content_layers = ['conv4_2']
style_layers = ['conv1_1', 'conv2_1', 'conv3_1']
alpha = 0.001
beta = [3e6, 3e5, 3e4, 3e3]
images = []
for i in xrange(4):
    images.append(reconstruction(content_layers, style_layers, alpha = alpha, beta = beta[i]))
    
titles = ['10^-9', '10^-8', '10^-7','10^-6']
plt.figure(figsize=(60,60))
for i in xrange(4):
    plt.subplot(1, 4, i+1)
    plt.ylabel('{0}'.format(style_layers[-1]))
    plt.title('{0}'.format(titles[i]))
    plt.imshow(reveal(images[i]))
plt.savefig('{0}.jpg'.format(style_layers[-1]))

In [0]:
content_layers = ['conv4_2']
style_layers = ['conv1_1', 'conv2_1', 'conv3_1','conv4_1']
alpha = 0.001
beta = [4e6, 4e5, 4e4, 4e3]
images = []
for i in xrange(4):
    images.append(reconstruction(content_layers, style_layers, alpha = alpha, beta = beta[i]))
    
titles = ['10^-9', '10^-8', '10^-7','10^-6']
plt.figure(figsize=(60,60))
for i in xrange(4):
    plt.subplot(1, 4, i+1)
    plt.ylabel('{0}'.format(style_layers[-1]))
    plt.title('{0}'.format(titles[i]))
    plt.imshow(reveal(images[i]))
plt.savefig('{0}.jpg'.format(style_layers[-1]))

In [0]:
content_layers = ['conv4_2']
style_layers = ['conv1_1', 'conv2_1', 'conv3_1','conv4_1','conv5_1]
alpha = 0.001
beta = [5e6, 5e5, 5e4, 5e3]
images = []
for i in xrange(4):
    images.append(reconstruction(content_layers, style_layers, alpha = alpha, beta = beta[i]))
                
titles = ['10^-9', '10^-8', '10^-7','10^-6']
plt.figure(figsize=(60,60))
for i in xrange(4):
    plt.subplot(1, 4, i+1)
    plt.ylabel('{0}'.format(style_layers[-1]))
    plt.title('{0}'.format(titles[i]))
    plt.imshow(reveal(images[i]))
plt.savefig('{0}.jpg'.format(style_layers[-1]))

In [0]:
content_layers = ['conv1_1', 'conv2_1', 'conv3_1','conv4_1','conv5_1']
style_layers = ['conv1_1']
alpha = 0.001
beta = 0
images = []
for i in xrange(5):
    images.append(reconstruction([content_layers[i]], style_layers, alpha = alpha, beta = beta))
    
plt.figure(figsize=(60,60))
for i in xrange(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(reveal(images[i]))
plt.savefig('{0}.jpg'.format('content reconstruction'))

In [0]:
content_layers = ['conv1_1', 'conv2_1', 'conv3_1','conv4_1','conv5_1']
style_layers = ['conv1_1']
alpha = 0.001
beta = 0
images = []
for i in xrange(5):
    images.append(reconstruction([content_layers[i]], style_layers, alpha = alpha, beta = beta))
    
plt.figure(figsize=(60,60))
for i in xrange(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(reveal(images[i])[150:300,350:500,:])
plt.savefig('{0}.jpg'.format('content reconstruction details'))
